In [1]:
import os, boto3, requests, io, json
from io import StringIO
from datetime import datetime, timedelta, timezone
import pandas as pd
from psycopg2 import pool, sql

In [2]:
with open("credentials/aws_keys.json") as f:
    keys = json.load(f)

os.environ["AWS_ACCESS_KEY_ID"] = keys["AWS_ACCESS_KEY_ID"]
os.environ["AWS_SECRET_ACCESS_KEY"] = keys["AWS_SECRET_ACCESS_KEY"]
os.environ["AWS_DEFAULT_REGION"] = keys["AWS_DEFAULT_REGION"]

In [3]:
s3 = boto3.client("s3")

bucket_name = "hab-ree-data-json"
file_key = "merged/ree-data_full.json"

response = s3.get_object(Bucket=bucket_name, Key=file_key)
data_json = json.loads(response["Body"].read())
print(f"Se cargaron {len(data_json)} registros")

Se cargaron 4276 registros


In [4]:

connection_pool = pool.SimpleConnectionPool(
    1, 10,  
    host="13.62.85.98",
    database="ree",
    user="postgres",
    password="postgres"
)

conn = connection_pool.getconn()

connection_pool.putconn(conn)


In [10]:
from psycopg2 import sql

table_name = "demanda_electrica"

columns = data_json[0].keys()

columns_with_types = ", ".join(f"{col} TEXT" for col in columns)  
create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} ({columns_with_types});"

conn = connection_pool.getconn()
cursor = conn.cursor()

cursor.execute(create_table_query)
conn.commit()
cursor.close()
connection_pool.putconn(conn)

OperationalError: connection to server at "13.62.85.98", port 5432 failed: FATAL:  too many connections for database "ree"


In [14]:
from psycopg2.extras import execute_values

# Conexión desde el pool
conn = connection_pool.getconn()
cursor = conn.cursor()
table_name = "demanda_electrica"

cols = list(data_json[0].keys())
insert_query = sql.SQL("INSERT INTO {} ({}) VALUES %s").format(
    sql.Identifier(table_name),
    sql.SQL(', ').join(map(sql.Identifier, cols))
)

# Convertir los datos en lista de listas
values = [[row[col] for col in cols] for row in data_json]

# Definir tamaño de bloque
chunk_size = 2000  

# Insertar por bloques
for i in range(0, len(values), chunk_size):
    chunk = values[i:i+chunk_size]
    execute_values(cursor, insert_query, chunk, page_size=10000)

conn.commit()
cursor.close()
connection_pool.putconn(conn)


InterfaceError: connection already closed

In [13]:
cursor.close() 
conn.close()